In [7]:
import h5py
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data

photons = h5py.File("./input/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5", "r")
electrons = h5py.File("./input/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5", "r")

pX_train = photons['X'][0:174300,:,:,:].reshape(174300,2,32,32)
pX_test = photons['X'][174300:249000,:,:,:].reshape(74700,2,32,32)
py_train = photons['y'][0:174300]
py_test = photons['y'][174300:249000]

eX_train = electrons['X'][0:174300,:,:,:].reshape(174300,2,32,32)
eX_test = electrons['X'][174300:249000,:,:,:].reshape(74700,2,32,32)
ey_train = electrons['y'][0:174300]
ey_test = electrons['y'][174300:249000]

X_train = np.concatenate((pX_train, eX_train))
X_test = np.concatenate((pX_test, eX_test))
y_train = np.concatenate((py_train, ey_train))
y_test = np.concatenate((py_test, ey_test))

tensor_train_x = torch.Tensor(X_train)
tensor_train_y = torch.Tensor(y_train)
tensor_test_x = torch.Tensor(X_test)
tensor_test_y = torch.Tensor(y_test)

# Hyperparameters
num_epochs = 3
num_classes = 1
batch_size = 300
learning_rate = 0.003

train_dataset = data.TensorDataset(tensor_train_x, tensor_train_y)
train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = data.TensorDataset(tensor_test_x, tensor_test_y)
test_dataloader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(2, 16, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 8, kernel_size=3, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7200, 1500)
        self.fc2 = nn.Linear(1500, 100)
        self.fc3 = nn.Linear(100, 10)
        self.fc4 = nn.Linear(10, 1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.relu(self.fc1(out))
        out = self.relu(self.fc2(out))
        out = self.relu(self.fc3(out))
        out = F.sigmoid(self.fc4(out))
        return out

model = ConvNet()

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_dataloader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dataloader):
        # Run the forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/3], Step [100/1162], Loss: 0.6880, Accuracy: 47.00%
Epoch [1/3], Step [200/1162], Loss: 0.6885, Accuracy: 44.67%
Epoch [1/3], Step [300/1162], Loss: 0.6676, Accuracy: 50.67%
Epoch [1/3], Step [400/1162], Loss: 0.6336, Accuracy: 52.33%
Epoch [1/3], Step [500/1162], Loss: 0.6575, Accuracy: 50.33%
Epoch [1/3], Step [600/1162], Loss: 0.6729, Accuracy: 51.00%
Epoch [1/3], Step [700/1162], Loss: 0.6520, Accuracy: 52.33%
Epoch [1/3], Step [800/1162], Loss: 0.6702, Accuracy: 50.00%
Epoch [1/3], Step [900/1162], Loss: 0.6361, Accuracy: 52.67%
Epoch [1/3], Step [1000/1162], Loss: 0.6593, Accuracy: 52.00%
Epoch [1/3], Step [1100/1162], Loss: 0.6430, Accuracy: 51.67%
Epoch [2/3], Step [100/1162], Loss: 0.6386, Accuracy: 52.67%
Epoch [2/3], Step [200/1162], Loss: 0.6613, Accuracy: 47.33%
Epoch [2/3], Step [300/1162], Loss: 0.6572, Accuracy: 45.67%
Epoch [2/3], Step [400/1162], Loss: 0.6090, Accuracy: 50.00%
Epoch [2/3], Step [500/1162], Loss: 0.6554, Accuracy: 47.33%
Epoch [2/3], Step [600